In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
from libsvm.svmutil import *
from libsvm.svm import *
import csv

In [2]:
def ReadCSV(filename):
    with open(filename, newline='') as csvfile:
        row = csv.reader(csvfile)
        data_list = [[float(ele) for ele in array] for array in row]
    
    data = np.array(data_list)
    row, col = data.shape
    return [data, row, col]

In [3]:
X_train, X_train_row, X_train_col = ReadCSV("X_train.csv")
Y_train, Y_train_row, Y_train_col = ReadCSV("Y_train.csv")
Y_train = Y_train.reshape(5000,)

X_test, X_test_row, X_test_col = ReadCSV("X_test.csv")
Y_test, Y_test_row, Y_test_col = ReadCSV("Y_test.csv")
Y_test = Y_test.reshape(2500, )

In [4]:
@jit
def LinearKernel(Xa, Xb):
    Xa = np.array(Xa)
    Xb = np.array(Xb)
    return Xa.T @ Xb

In [5]:
@jit
def RBFkernel(Xa, Xb):
    gamma = 1/784
    difference = np.array([Xa[i] - Xb[i] for i in range(len(Xa))])
    return np.exp(-1 * gamma * (difference.T @ difference))

In [6]:
@jit
def GetDataByKernel(X_train):
    length = X_train.shape[0]
    output = []
    for i in range(length):
        tmp = []
        tmp.append(i + 1)
        for j in range(length):
            val = LinearKernel(X_train[i], X_train[j]) + RBFkernel(X_train[i], X_train[j])
            tmp.append(val)
        output.append(tmp)
    return np.array(output)

In [13]:
a = np.array([X_train[i] for i in range(0, 5000, 10)])
a_y = np.array([Y_train[i] for i in range(0, 5000, 10)])

In [15]:
b = np.array([X_test[i] for i in range(0, 2500, 10)])
b_y = np.array([Y_test[i] for i in range(0, 2500, 10)])

In [16]:
new_train = GetDataByKernel(a)

In [17]:
new_test = GetDataByKernel(b)

In [18]:
model = svm_train(a_y, new_train, '-t 4')
label, acc, val = svm_predict(b_y, new_test, model)

Accuracy = 18% (45/250) (classification)


In [ ]:
new_test.shape

In [ ]:
label, acc, val = svm_predict(Y_test, new_test, model)